In [83]:
# imports
import numpy as np
import pandas as pd

import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt

# Dataset Import

In [84]:
tv_shows = pd.read_csv('tv_shows.csv')
tv_shows.head()

Unnamed: 0  ID             Title  Year  Age    IMDb Rotten Tomatoes  \
0           0   1      Breaking Bad  2008  18+  9.4/10         100/100   
1           1   2   Stranger Things  2016  16+  8.7/10          96/100   
2           2   3   Attack on Titan  2013  18+  9.0/10          95/100   
3           3   4  Better Call Saul  2015  18+  8.8/10          94/100   
4           4   5              Dark  2017  16+  8.8/10          93/100   

   Netflix  Hulu  Prime Video  Disney+  Type  
0        1     0            0        0     1  
1        1     0            0        0     1  
2        1     1            0        0     1  
3        1     0            0        0     1  
4        1     0            0        0     1

In [ ]:
tv_shows.drop_duplicates(subset='Title', keep='first',inplace=True)

In [ ]:
tv_shows['Rotten Tomatoes'] = tv_shows['Rotten Tomatoes'].fillna('0.0/10').str.replace('/100', '')
tv_shows['Rotten Tomatoes'] = pd.to_numeric(tv_shows['Rotten Tomatoes'])
tv_shows['IMDb'] = tv_shows['IMDb'].fillna('0.0/10').str.replace('/10', '')
tv_shows['IMDb'] = pd.to_numeric(tv_shows['IMDb'])

tv_shows['IMDb'] = tv_shows['IMDb'].fillna(0)
tv_shows['IMDb'] = tv_shows['IMDb']*10
tv_shows['IMDb'] = tv_shows['IMDb'].astype('int')

In [ ]:
tv_shows_long=pd.melt(tv_shows[['Title','Netflix','Hulu','Disney+', 'Prime Video']],id_vars=['Title'], var_name='StreamingOn', value_name='Present')
tv_shows_long = tv_shows_long[tv_shows_long['Present'] == 1]
tv_shows_long.drop(columns=['Present'],inplace=True)

In [ ]:
tv_shows_combined = tv_shows_long.merge(tv_shows, on='Title', how='inner')
tv_shows_combined.drop(columns = ['Unnamed: 0','Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Type'], inplace=True)

In [ ]:
tv_shows_both_ratings = tv_shows_combined[(tv_shows_combined.IMDb > 0) & tv_shows_combined['Rotten Tomatoes'] > 0]
tv_shows_combined.groupby('StreamingOn').Title.count().plot(kind='bar')

In [ ]:
figure = []
figure.append(px.violin(tv_shows_both_ratings, x = 'StreamingOn', y = 'IMDb', color='StreamingOn'))
figure.append(px.violin(tv_shows_both_ratings, x = 'StreamingOn', y = 'Rotten Tomatoes', color='StreamingOn'))
fig = make_subplots(rows=2, cols=4, shared_yaxes=True)

for i in range(2):
    for j in range(4):
        fig.add_trace(figure[i]['data'][j], row=i+1, col=j+1)

fig.update_layout(autosize=False, width=800, height=800)        
fig.show()